这是一个用于临时测试代码的ipynb文件

In [1]:
import os
import re
import string
import torch
from torch import nn
from torch.nn import functional as F
if not "CHDIR_FLAG" in dir():
    os.chdir("../")
    CHDIR_FLAG = True

from transformers import AutoTokenizer, AutoModelForCausalLM, Qwen2Model, Qwen2ForCausalLM

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

from src.module import (
    SkipLayerQwen2Model, 
    SkipLayerQwen2ForCausalLM,
    ParallelQwen2Model,
    ParallelQwen2ForCausalLM,
)

from src.pipelines.analysis import horizontal_comparison_of_forward_hook, vertical_comparison_of_forward_hook, skip_layer_generation, easy_skip_layer_generation

In [ ]:
from transformers import DeepseekForCausalLM

In [2]:
model_home = r"D:\resource\model\huggingface"
model_names = [
    r"Qwen\Qwen2.5-0.5B-Instruct",
    r"deepseek-ai\DeepSeek-R1-Distill-Qwen-1.5B",
]

# 测试 horizontal_comparison_of_forward_hook

In [ ]:
forward_hook_module_names = \
    [f"model.layers[{i}].self_attn.q_proj" for i in range(24)] + \
    [f"model.layers[{i}].self_attn.k_proj" for i in range(24)] + \
    [f"model.layers[{i}].self_attn.v_proj" for i in range(24)]

hook_data_path_1_1 = r"./results/strawberry-1/fhook+Qwen2.5-0.5B-Instruct+False.pt"
hook_data_path_1_2 = r"./results/strawberry-1/fhook+Qwen2.5-0.5B-Instruct+True.pt"
hook_data_path_2_1 = r"./results/strawberry-2/fhook+Qwen2.5-0.5B-Instruct+False.pt"
hook_data_path_2_2 = r"./results/strawberry-2/fhook+Qwen2.5-0.5B-Instruct+True.pt"

table_path_1_1 = r"./results/strawberry-1/decode+Qwen2.5-0.5B-Instruct+False.csv"
table_path_1_2 = r"./results/strawberry-1/decode+Qwen2.5-0.5B-Instruct+True.csv"
table_path_2_1 = r"./results/strawberry-2/decode+Qwen2.5-0.5B-Instruct+False.csv"
table_path_2_2 = r"./results/strawberry-2/decode+Qwen2.5-0.5B-Instruct+True.csv"

In [ ]:
horizontal_comparison_of_forward_hook(
    hook_datas = None,
    hook_data_paths = [hook_data_path_1_2, hook_data_path_2_2],
    hook_module_names = forward_hook_module_names[:],
    hook_module_name_suffixes = ["q_proj", "k_proj", "v_proj"],
    comparison_index = ["mean_diff", "max_diff", "corr", "sim", "robust_corr", "robust_sim"],
    max_length = 4,
    outlier_ratio = .1
)

In [ ]:
horizontal_comparison_of_forward_hook(
    hook_datas = None,
    hook_data_paths = [hook_data_path_1_2, hook_data_path_2_2],
    hook_module_names = forward_hook_module_names[:],
    hook_module_name_suffixes = ["q_proj", "k_proj", "v_proj"],
    comparison_index = ["mean_diff", "max_diff", "corr", "sim"],
    max_length = 4,
    outlier_ratio = .0
)

# 测试 vertical_comparison_of_forward_hook

观察结论：
- 第0层输入输出虽然diff很小，但是相关系数非常低，说明语义在这一层做了一个很大的变换，可能是旋转之类的，所以第0层非常重要
- 第1层基本上的相似度也是显著低于其余层的

In [ ]:
hook_data_paths = [
    r"./results/strawberry-X-4/fhook+Qwen2.5-0.5B-Instruct+True-0.pt",
    r"./results/longlong-1/fhook+Qwen2.5-0.5B-Instruct+True-0.pt",
    r"./results/prime-1/fhook+Qwen2.5-0.5B-Instruct+True-0.pt",
    r"./results/table-1/fhook+Qwen2.5-0.5B-Instruct+True-1.pt",
    r"./results/strawberry-X-3/fhook+DeepSeek-R1-Distill-Qwen-1.5B+True-0.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+False-0.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+False-1.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+False-2.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+False-3.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+True-0.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+True-1.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+True-2.pt",
    r"./results/selfattn-1/fhook+Qwen2.5-0.5B-Instruct+True-3.pt",
]

## Strawberry

In [ ]:
vertical_comparison_of_forward_hook(
    hook_data = None,
    hook_data_path = hook_data_paths[0], # prime-true
    hook_module_names = [f"model.layers[{i}]" for i in range(24)],
    comparison_index = ["mean_diff", "max_diff", "sim", "robust_sim"],
    max_length = 16,
    figure_size = 5,
    watched_module_names = [f"model.layers[{i}]" for i in [2]],
    outlier_ratio = .1
)

In [ ]:
vertical_comparison_of_forward_hook(
    hook_data = None,
    hook_data_path = hook_data_paths[0], # prime-true
    hook_module_names = [f"model.layers[{i}]" for i in range(24)],
    comparison_index = ["mean_diff", "max_diff", "sim", "robust_sim"],
    max_length = 16,
    figure_size = 5,
    watched_module_names = [f"model.layers[{i}]" for i in []],
    outlier_ratio = .05
)

## longlong

In [ ]:
vertical_comparison_of_forward_hook(
    hook_data = None,
    hook_data_path = hook_data_paths[1], # prime-true
    hook_module_names = [f"model.layers[{i}]" for i in range(24)],
    comparison_index = ["mean_diff", "max_diff", "corr", "sim"],
    max_length = 4,
    figure_size = 5,
    watched_module_names = [f"model.layers[{i}]" for i in [2, 3, 5, 20, 21, 22, 23]],
)

## prime

In [ ]:
vertical_comparison_of_forward_hook(
    hook_data = None,
    hook_data_path = hook_data_paths[2],
    hook_module_names = [f"model.layers[{i}]" for i in range(24)],
    comparison_index = ["mean_diff", "max_diff", "corr", "sim"],
    max_length = 16,
    figure_size = 5,
    watched_module_names = [f"model.layers[{i}]" for i in [2, 3, 5, 20, 21, 22, 23]],
)

## table

In [ ]:
vertical_comparison_of_forward_hook(
    hook_data = None,
    hook_data_path = hook_data_paths[3], # prime-true
    hook_module_names = [f"model.layers[{i}]" for i in range(24)],
    comparison_index = ["mean_diff", "max_diff", "corr", "sim"],
    max_length = 4,
    figure_size = 5,
    watched_module_names = [f"model.layers[{i}]" for i in [2, 3, 5, 20, 21, 22, 23]],
)

# 测试skip_layer封装

- 目前这两种写法对past_key_values以及hook的影响：

  - past_key_values永远是num_hidden_layers的长度（DynamicCache），然后被跳过的层的past_key_values是两个(None, None)，这两个都一样
  - hook是有区别的，forward_hook_module_names的写法是要区别的：
    - 第一种方法（每次推理需要重新加载模型），forward_hook_module_names永远是`["model.layers[0]", ..., model.layers[num_hidden_layers-1]]`，但是被跳过的那几层的hook是没有数据的（即args, kwargs, outputs都是空列表）
    - 第二种方法（每次推理无需重新加载模型），forward_hook_module_names是不包含skip_layer_ids的层
    - 原因是第一种方法其实只是在forward函数中重新调整的`self.layers`，整体上模型还是那么多层，但是后者则是直接把整个模型的层给扔了，整个推理完才加回来的，所以forward_hook_module_names的layer_id必须是不能包含skip_layer_ids，否则hook会报错，根本找不到那些层

## 使用重写的模型类（skip不同的层推理需要重新加载模型）

In [ ]:
from transformers import Qwen2Model, Qwen2ForCausalLM
model_path = os.path.join(model_home, model_names[0]) # Qwen2-0.5B
prompt = "解方程：x^2 - 3x + 2 = 0"
max_length = 32
tokenizer = AutoTokenizer.from_pretrained(model_path)
results = skip_layer_generation(
    SkipLayerModelForCausalLM = SkipLayerQwen2ForCausalLM,
    model_name_or_path = model_path,
    tokenizer = tokenizer,
    prompt = prompt,
    max_length = max_length,
    skip_layer_ids = [0],
    use_kv_cache = True,
    forward_hook_module_names = None,
    backward_hook_module_names = None,
)
print(results["text"])

In [ ]:
from transformers import Qwen2Model, Qwen2ForCausalLM
model_path = os.path.join(model_home, model_names[0]) # Qwen2-0.5B
prompt = "解方程：x^2 - 3x + 2 = 0"
max_length = 32
tokenizer = AutoTokenizer.from_pretrained(model_path)
results = skip_layer_generation(
    SkipLayerModelForCausalLM = SkipLayerQwen2ForCausalLM,
    model_name_or_path = model_path,
    tokenizer = tokenizer,
    prompt = prompt,
    max_length = max_length,
    skip_layer_ids = [0],
    use_kv_cache = True,
    forward_hook_module_names = ["model.layers[1]"],
    backward_hook_module_names = None,
)
print(results["text"])

In [ ]:
from transformers import Qwen2Model, Qwen2ForCausalLM
model_path = os.path.join(model_home, model_names[0]) # Qwen2-0.5B
prompt = "很久很久以前"
max_length = 32
tokenizer = AutoTokenizer.from_pretrained(model_path)
results = skip_layer_generation(
    SkipLayerModelForCausalLM = SkipLayerQwen2ForCausalLM,
    model_name_or_path = model_path,
    tokenizer = tokenizer,
    prompt = prompt,
    max_length = max_length,
    skip_layer_ids = [0, 2],
    use_kv_cache = True,
    forward_hook_module_names = None,
    backward_hook_module_names = None,
)
print(results["text"])

## 使用easy模型（skip不同的层推理不需要重新加载模型）

In [3]:
model_path = os.path.join(model_home, model_names[0]) # Qwen2-0.5B
prompt = "很久很久以前"
max_length = 32

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [4]:
for name, param in model.named_parameters():
    print(name, param.device)

model.embed_tokens.weight cpu
model.layers.0.self_attn.q_proj.weight cpu
model.layers.0.self_attn.q_proj.bias cpu
model.layers.0.self_attn.k_proj.weight cpu
model.layers.0.self_attn.k_proj.bias cpu
model.layers.0.self_attn.v_proj.weight cpu
model.layers.0.self_attn.v_proj.bias cpu
model.layers.0.self_attn.o_proj.weight cpu
model.layers.0.mlp.gate_proj.weight cpu
model.layers.0.mlp.up_proj.weight cpu
model.layers.0.mlp.down_proj.weight cpu
model.layers.0.input_layernorm.weight cpu
model.layers.0.post_attention_layernorm.weight cpu
model.layers.1.self_attn.q_proj.weight cpu
model.layers.1.self_attn.q_proj.bias cpu
model.layers.1.self_attn.k_proj.weight cpu
model.layers.1.self_attn.k_proj.bias cpu
model.layers.1.self_attn.v_proj.weight cpu
model.layers.1.self_attn.v_proj.bias cpu
model.layers.1.self_attn.o_proj.weight cpu
model.layers.1.mlp.gate_proj.weight cpu
model.layers.1.mlp.up_proj.weight cpu
model.layers.1.mlp.down_proj.weight cpu
model.layers.1.input_layernorm.weight cpu
model.lay

In [ ]:
results = easy_skip_layer_generation(
    model = model,
    tokenizer = tokenizer,
    prompt = prompt,
    max_length = max_length,
    skip_layer_ids = [0, 2],
    use_kv_cache = True,
    forward_hook_module_names = None,
    backward_hook_module_names = None,
)
results["text"]

In [ ]:
results = easy_skip_layer_generation(
    model = model,
    tokenizer = tokenizer,
    prompt = "解方程：x^2 - 3x + 2 = 0",
    max_length = max_length,
    skip_layer_ids = [0],
    use_kv_cache = False,
    forward_hook_module_names = None,
    backward_hook_module_names = None,
)
results["text"]

In [ ]:
results = easy_skip_layer_generation(
    model = model,
    tokenizer = tokenizer,
    prompt = "解方程：x^2 - 3x + 2 = 0",
    max_length = max_length,
    skip_layer_ids = [0],
    use_kv_cache = True,
    forward_hook_module_names = None,
    backward_hook_module_names = None,
)
results["text"]

In [ ]:
results = easy_skip_layer_generation(
    model = model,
    tokenizer = tokenizer,
    prompt = "解方程：x^2 - 3x + 2 = 0",
    max_length = max_length,
    skip_layer_ids = [0],
    use_kv_cache = True,
    forward_hook_module_names = ["model.layers[0]"],
    backward_hook_module_names = None,
)
results["text"]

In [ ]:
results["forward_hook_data"]

# 测试Qwen模型并行

In [ ]:
# -*- coding: utf8 -*-
# @author: caoyang
# @email: caoyang@stu.sufe.edu.cn

import torch
from transformers import Qwen2Model, Qwen2ForCausalLM
from transformers.cache_utils import Cache, DynamicCache

# Overwrite according to /transformers/models/qwen2/modeling_qwen2.py
# Version transformers 4.56.3
class ParallelQwen2Model(Qwen2Model):
    def __init__(self, config, n_cuda = 2, **kwargs):
        super().__init__(config, **kwargs)
        self._module_to_device()

    def _module_to_device(self):
        self.embed_tokens = self.embed_tokens.to("cuda:0")
        self.norm = self.norm.to(f"cuda:{self.n_cuda - 1}")
        n_layers = len(self.layers)
        self.layer_to_device = dict()
        for layer_id in range(n_layers):
            for device_id in range(self.n_cuda):
                if layer_id <= (device_id + 1) * n_layers // self.n_cuda:
                    self.layers[layer_id] = self.layers[layer_id].to(f"cuda:{device_id}")
                    self.layer_id_to_device[layer_id] = device_id
                    break

    def forward(self,
                input_ids = None,
                attention_mask = None,
                position_ids= None,
                past_key_values = None,
                inputs_embeds = None,
                use_cache = None,
                cache_position = None,
                **kwargs,
                ):
        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError("You must specify exactly one of input_ids or inputs_embeds")
        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids)
        if use_cache and past_key_values is None:
            past_key_values = DynamicCache(config=self.config)
        if cache_position is None:
            past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
            cache_position = torch.arange(
                past_seen_tokens, 
                past_seen_tokens + inputs_embeds.shape[1], 
                device = inputs_embeds.device,
            )
        if position_ids is None:
            position_ids = cache_position.unsqueeze(0)
        # It may already have been prepared by e.g. `generate`
        if not isinstance(causal_mask_mapping := attention_mask, dict):
            # Prepare mask arguments
            mask_kwargs = {
                "config": self.config,
                "input_embeds": inputs_embeds,
                "attention_mask": attention_mask,
                "cache_position": cache_position,
                "past_key_values": past_key_values,
                "position_ids": position_ids,
            }
            # Create the masks
            causal_mask_mapping = {
                "full_attention": create_causal_mask(**mask_kwargs),
            }
            # The sliding window alternating layers are not always activated depending on the config
            if self.has_sliding_layers:
                causal_mask_mapping["sliding_attention"] = create_sliding_window_causal_mask(**mask_kwargs)
        hidden_states = inputs_embeds
        # create position embeddings to be shared across the decoder layers
        position_embeddings = self.rotary_emb(hidden_states, position_ids)
        # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
        for layer_id, decoder_layer in enumerate(self.layers[: self.config.num_hidden_layers]):
            current_device_id = self.layer_to_device[layer_id]
            hidden_states = decoder_layer(
                hidden_states,
                attention_mask=causal_mask_mapping[decoder_layer.attention_type],
                position_ids=position_ids,
                past_key_values=past_key_values,
                use_cache=use_cache,
                cache_position=cache_position,
                position_embeddings=position_embeddings,
                **kwargs,
            )
            if layer_id < self.config.num_hidden_layers - 1:
                next_device_id = self.layer_to_device[layer_id]
                if not current_device_id == next_device_id:
                    next_device_name = f"cuda:{next_device_id}"
                    hidden_states = hidden_states.to(next_device_name)
                    attention_mask = causal_mask_mapping[decoder_layer.attention_type].to(next_device_name)
                    position_ids = position_ids.to(next_device_name)
                    cache_position = cache_position.to(next_device_name)
                    # Deal with KV-Cache
                    if past_key_values is not None:
                        for i in range(past_key_values):
                            if past_key_values[i][0] is not None:
                                past_key_values[i][0] = past_key_values[i][0].to(next_device_name)
                            if past_key_values[i][1] is not None:
                                past_key_values[i][1] = past_key_values[i][1].to(next_device_name)
        # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        hidden_states = self.norm(hidden_states)
        return BaseModelOutputWithPast(
            last_hidden_state = hidden_states,
            past_key_values = past_key_values if use_cache else None,
        )


In [ ]:
model_path = r"D:\resource\model\huggingface\Qwen\Qwen2.5-0.5B-Instruct"
model = ParallelQwen2Model.from_pretrained(model_path)

In [ ]:
for name, param in model.named_parameters():
    print(name, param.device)